# Determine Point Totals and Success of Optimal Rosters based on Expert Predictions

## read in expert predictions for weeks 1-10 2020

In [1]:
using JuMP, Gurobi, CSV, DataFrames

In [2]:
all_data = DataFrame(CSV.File("output/expert_projections_with_DKP_2020.csv"))
;

## read in point total required to make money in a given week (https://rotogrinders.com/resultsdb)

In [3]:
money_mat = [
    2020 1 144.3
    2020 2 132.6
    2020 3 135.8
    2020 4 136.1
    2020 5 145.04
    2020 6 139.94
    2020 7 177.6
    2020 8 121.02
    2020 9 136.34
    2020 10 118.5
]
;

## find optimal roster and store results

In [4]:
extras = 9 # number of extra suboptimal rosters to create
results = Array{Any}(undef,length(1:10),5+extras) # to store results
row = 1 #used to iterate over when populating "results" array
eps = 1 #degree to which objective value must decrease when finding suboptimal roster. 

#perform analysis in weeks 1-10 for 2020.
for week in 1:10
    println(week)
    #subset to week being analyzed for both predictions and money line
    df = all_data[(all_data[:,:week] .== week) .& (all_data[:,:year] .== 2020),:]
    money = money_mat[(money_mat[:,1] .== 2020) .& (money_mat[:,2] .== week),3]

    #get optimal roster given expert predictions
    #optimization model
    model = Model(Gurobi.Optimizer)
    set_optimizer_attributes(model, "OutputFlag"=>0)

    num_players = size(df)[1]
    # Insert variables and constraints
    @variable(model,z[i=1:num_players],Bin)
    @constraint(model,sum(z[i]*df[i,:DKSalary] for i=1:num_players) <= 50000)
    @constraint(model,sum(z[i]*df[i,:Pos_Def] for i=1:num_players) == 1)
    @constraint(model,sum(z[i]*df[i,:Pos_QB] for i=1:num_players) == 1)
    @constraint(model,sum(z[i]*df[i,:Pos_TE] for i=1:num_players) >= 1)
    @constraint(model,sum(z[i]*df[i,:Pos_RB] for i=1:num_players) >= 2)
    @constraint(model,sum(z[i]*df[i,:Pos_WR] for i=1:num_players) >= 3)
    @constraint(model,sum(z[i]*(df[i,:Pos_TE] + df[i,:Pos_RB] + df[i,:Pos_WR]) for i=1:num_players) == 7)
    @objective(model,Max, sum(z[i]*df[i,:projection] for i=1:num_players))

    # Optimize
    optimize!(model)
    obj = objective_value(model)
    #actual points from optimal roster (chosen based on predicted points)
    points = sum(df[value.(z) .== 1,:DKP])
    results[row,1] = 2020
    results[row,2] = week
    results[row,3] = "expert"
    results[row,4] = money[1]
    results[row,5] = points

    #create additional suboptimal rosters
    for i in 1:extras
        @constraint(model,sum(z[i]*df[i,:projection] for i=1:num_players) <= obj - eps)
        optimize!(model)
        obj = objective_value(model)
        #actual points from optimal roster (chosen based on predicted points)
        points = sum(df[value.(z) .== 1,:DKP])
        results[row,i+5] = points
    end
    row = row + 1
end

1
Academic license - for non-commercial use only
Academic license - for non-commercial use only
2
Academic license - for non-commercial use only
Academic license - for non-commercial use only
3
Academic license - for non-commercial use only
Academic license - for non-commercial use only
4
Academic license - for non-commercial use only
Academic license - for non-commercial use only
5
Academic license - for non-commercial use only
Academic license - for non-commercial use only
6
Academic license - for non-commercial use only
Academic license - for non-commercial use only
7
Academic license - for non-commercial use only
Academic license - for non-commercial use only
8
Academic license - for non-commercial use only
Academic license - for non-commercial use only
9
Academic license - for non-commercial use only
Academic license - for non-commercial use only
10
Academic license - for non-commercial use only
Academic license - for non-commercial use only


## save results for further analysis in R

In [5]:
CSV.write("output/optimization_results.csv",DataFrame(results))
;